In [17]:
#Import libraries

import pandas as pd
import seaborn as sns
import requests
import json
import schedule
from datetime import datetime, timedelta, time
import time

In [18]:
# Check for total count of bike stations

bikePoints = requests.get("https://api.tfl.gov.uk/BikePoint?app_key=fe64e060ff7e4a8b9c3cc783588b6601")
bikePoints = bikePoints.json()
len(bikePoints)

787

In [19]:
# Creating a data-frame to store the latest bike station IDs, station names, co-ordinates
current_log = list()

for station in range(len(bikePoints)):
    stationTflID = bikePoints[station]['id']
    stationName = bikePoints[station]['commonName']
    stationLat = bikePoints[station]['lat']
    stationLon = bikePoints[station]['lon']
    stationRecord = [stationTflID, stationName, stationLat, stationLon]
    current_log.append(stationRecord)
    
dfStationLocation = pd.DataFrame(current_log, columns = ['Station ID', 'Station Name', 'Latitude', 'Longitude'])
dfStationLocation.head()


,Station ID,Station Name,Latitude,Longitude
0,BikePoints_103,"Vicarage Gate, Kensington",51.504723,-0.192538
1,BikePoints_104,"Crosswall, Tower",51.511594,-0.077121
2,BikePoints_105,"Westbourne Grove, Bayswater",51.515529,-0.190240
3,BikePoints_106,"Woodstock Street, Mayfair",51.514105,-0.147301
4,BikePoints_107,"Finsbury Leisure Centre, St. Luke's",51.526008,-0.096317


In [44]:
# Function for requesting bike station information
# Insert your API key in the starred 
def pullAvailability(DocksLog):
    curr_time  = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    bikePoints = requests.get("https://api.tfl.gov.uk/BikePoint?app_key=**INSERT YOUR KEY HERE**")
    bikePoints = bikePoints.json()
    
    for station in range(len(bikePoints)):
        stationTflID = bikePoints[station]['id']
        stationName = bikePoints[station]['commonName']
        NbBikes      = int((bikePoints[station]['additionalProperties'][6]['value']).strip() or -999 )
        NbEmptyDocks = int((bikePoints[station]['additionalProperties'][7]['value']).strip() or -999 ) # catered for empty string problem
        NbTotDocks   = int((bikePoints[station]['additionalProperties'][8]['value']).strip() or -999 )
        BrokenDocks  = NbTotDocks - ( NbBikes + NbEmptyDocks ) 
        stationAvailRecord = [curr_time, stationTflID, stationName, NbBikes, NbEmptyDocks, NbTotDocks, BrokenDocks]
        DocksLog.append(stationAvailRecord)

In [45]:
# Function to record all london bike stations availability, for a given period
def RecordBikeAvailabilityTest(StartTime,EndTime):
    
    #Initalisations
    DiffInTime = 99
    DiffInStartTime = 99
    TempLog  = list()
    CurrDateTime = datetime.now()
    CurrDateTime_Year   = str(CurrDateTime.year)
    CurrDateTime_Day    = str(CurrDateTime.day)
    CurrDateTime_Month  = str(CurrDateTime.month)

    StartDateTime  = datetime.strptime('{Time} {Day}-{Month}-{Yr}'.format(Time  = StartTime, 
                                                              Day   = CurrDateTime_Day,
                                                              Month = CurrDateTime_Month,
                                                              Yr    = CurrDateTime_Year),
                                    '%H:%M %d-%m-%Y')
    
    EndDateTime  = datetime.strptime('{Time} {Day}-{Month}-{Yr}'.format(Time  = EndTime, 
                                                              Day   = CurrDateTime_Day,
                                                              Month = CurrDateTime_Month,
                                                              Yr    = CurrDateTime_Year),
                                    '%H:%M %d-%m-%Y')
    
    # Wait before start time is reached
    while DiffInStartTime >= 0:
        #print('Part1')
        time.sleep(5)
        CurrDateTime = datetime.now()
        TempDifference = StartDateTime-CurrDateTime
        DiffInStartTime = TempDifference.days*24 + TempDifference.seconds/(60*60)
        print(DiffInStartTime)
        
    
    # Check schedule to run, while loop ends when current time has passed end time
    while DiffInTime >= 0:
        #print('Part2')
        pullAvailability(TempLog)
        #print('Part3')
        time.sleep(120)
        CurrDateTime = datetime.now()
        TempDifference = EndDateTime-CurrDateTime
        DiffInTime = TempDifference.days*24 + TempDifference.seconds/(60*60)
        
        
    #Store the collected data in a dataframe
    dfStationAvailability = pd.DataFrame(TempLog, 
                                         columns = ['DateTime',
                                                    'Station ID',
                                                    'Station Name',
                                                    'NbBikes',
                                                    'NbEmptyDocks',
                                                    'NbTotDocks',
                                                    'NbBrokenDocks' ])
    
    dfStationAvailability.to_csv('/Users/anthonyng/Desktop/DataScience/20-02-2022-PM.csv') 

    return dfStationAvailability

In [46]:
RecordBikeAvailabilityTest("23:55","23:57")

-0.018055555555555713


,DateTime,Station ID,Station Name,NbBikes,NbEmptyDocks,NbTotDocks,NbBrokenDocks
0,20/01/2022 23:56:04,BikePoints_103,"Vicarage Gate, Kensington",6,12,18,0
1,20/01/2022 23:56:04,BikePoints_104,"Crosswall, Tower",12,22,34,0
2,20/01/2022 23:56:04,BikePoints_105,"Westbourne Grove, Bayswater",17,7,26,2
3,20/01/2022 23:56:04,BikePoints_106,"Woodstock Street, Mayfair",6,14,21,1
4,20/01/2022 23:56:04,BikePoints_107,"Finsbury Leisure Centre, St. Luke's",13,7,20,0
...,...,...,...,...,...,...,...
782,20/01/2022 23:56:04,BikePoints_817,"Riverlight South, Nine Elms",20,3,26,3
783,20/01/2022 23:56:04,BikePoints_818,"Westminster Pier, Westminster",9,17,27,1
784,20/01/2022 23:56:04,BikePoints_819,"Belvedere Road 2, South Bank",34,4,38,0
785,20/01/2022 23:56:04,BikePoints_820,"Victory Place, Walworth",2,22,24,0
